In [1]:
from utils.BoW_data import load_dataframe as ld

In [2]:
mlb_tags_train, mlb_words_train, df_train = ld()

In [3]:
mlb_words_train.classes_[0]

'11'

In [4]:
df_train.head()

,image_file,resnet_vector,descriptions,tags,tags_values,tags_vec,word_list,word_vector
0,images_train/5373.jpg,"[-0.8994496464729309, -0.9304700493812561, -2....","[a red train is docked at the station, Several...","[vehicle:train, person:person, indoor:clock, a...","[train, person, clock, handbag]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[red, train, dock, station]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,images_train/984.jpg,"[-1.3469539880752563, -3.1194605827331543, -0....",[A man with blue jersey holding a baseball bat...,"[person:person, sports:baseball bat]","[person, baseball bat]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[man, blue, jersey, hold, basebal, bat]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,images_train/7127.jpg,"[-3.44549822807312, -1.5245732069015503, -1.00...",[A kitchen decorated in red and white with acc...,"[appliance:refrigerator, appliance:oven, appli...","[refriger, oven, sink, cup, cake, vase]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[kitchen, decor, red, white, accessori]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,images_train/9609.jpg,"[1.1146496534347534, -2.1671018600463867, 0.09...",[A black and white dog chasing sheep in a fiel...,"[animal:dog, animal:sheep]","[dog, sheep]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[black, white, dog, chase, sheep, field]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,images_train/5293.jpg,"[1.6026496887207031, -1.5058174133300781, 3.02...",[Two bears with their mouths open in the water...,[animal:bear],[bear],"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[two, bear, mouth, open, water]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
_,_, df_test = ld('test')

In [6]:
df_test.head()

,image_file,resnet_vector,descriptions,tags,tags_values,tags_vec,word_list
0,images_test/152.jpg,"[-0.148756742477417, -0.4813389182090759, -0.6...","[A sign sitting above a store front entrance.,...",[person:person],[person],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[sign, sit, store, front, entranc]"
1,images_test/901.jpg,"[-2.7743351459503174, -2.609677791595459, -2.2...",[A steam locamotive passes by some houses on a...,[indoor:clock],[clock],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[steam, locamot, pass, hous, track]"
2,images_test/1609.jpg,"[-4.7030134201049805, -3.1210086345672607, -0....",[a woman with purple hair is taking a picture ...,"[animal:dog, animal:horse, person:person]","[dog, hors, person]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[woman, purpl, hair, take, pictur]"
3,images_test/501.jpg,"[-4.7203192710876465, -3.283935546875, -3.4253...",[A GREEN AND BROWN SUITCASE LYING UP ON A LEDG...,[furniture:toilet],[toilet],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[green, brown, suitcas, lie, ledg]"
4,images_test/517.jpg,"[-2.5421407222747803, 0.7139833569526672, -4.4...",[A desk with a small white computer set up on ...,"[animal:cat, kitchen:bowl, furniture:bed]","[cat, bowl, bed]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, ...","[desk, small, white, comput, set]"


In [7]:
from sklearn.naive_bayes import MultinomialNB

In [8]:
num_classes_tags = len(mlb_tags_train.classes_)
num_classes_words = len(mlb_words_train.classes_)

In [9]:
def word_list_to_vector(mlb,word_list,num_classes):
    r = [0]*num_classes
    for i,w in enumerate(mlb.classes_):
        if w in word_list:
            r[i] = 1
    return r

In [10]:
def tags_list_to_vector(mlb,tags_list,num_classes):
    r = [0]*num_classes
    for i,t in enumerate(mlb.classes_):
        if t in tags_list:
            r[i] = 1
    return r

In [11]:
test_tags = df_test['tags_values'].apply(lambda x: tags_list_to_vector(mlb_tags_train,x,num_classes_tags))
test_vectors = df_test['word_list'].apply(lambda x: word_list_to_vector(mlb_words_train,x,num_classes_words))

In [12]:
clf = MultinomialNB()
x = df_train['word_vector'].tolist()
y = df_train['image_file'].tolist()
clf.fit(x, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
train_tags = df_train['tags_vec'].tolist()
test_tags = list(zip(df_test['image_file'].tolist(),df_test['tags_vec'].tolist()))

In [14]:
def euclidean_distance(v1,v2):
    sum = 0
    for x,y in zip(v1,v2):
        sum += (x-y)**2
    return sum**0.5

In [15]:
def nearest20(training_tag,test_tags):
    nearest_test_neighbors = []
    for t in test_tags:
        nearest_test_neighbors.append((t[0],euclidean_distance(training_tag,t[1])))
    nearest_test_neighbors.sort(key=lambda x: x[1])
    return nearest_test_neighbors[:20]

In [16]:
predictions = df_train['tags_vec'].apply(lambda tags: nearest20(tags,test_tags))

In [17]:
print(predictions[1])

[('images_test/906.jpg', 0.0), ('images_test/1568.jpg', 0.0), ('images_test/28.jpg', 0.0), ('images_test/1400.jpg', 0.0), ('images_test/152.jpg', 1.0), ('images_test/1722.jpg', 1.0), ('images_test/1115.jpg', 1.0), ('images_test/1074.jpg', 1.0), ('images_test/1350.jpg', 1.0), ('images_test/897.jpg', 1.0), ('images_test/1535.jpg', 1.0), ('images_test/1135.jpg', 1.0), ('images_test/1520.jpg', 1.0), ('images_test/1224.jpg', 1.0), ('images_test/1036.jpg', 1.0), ('images_test/1610.jpg', 1.0), ('images_test/1406.jpg', 1.0), ('images_test/1218.jpg', 1.0), ('images_test/1217.jpg', 1.0), ('images_test/1725.jpg', 1.0), ('images_test/684.jpg', 1.0), ('images_test/512.jpg', 1.0), ('images_test/1974.jpg', 1.0), ('images_test/522.jpg', 1.0), ('images_test/141.jpg', 1.0), ('images_test/1633.jpg', 1.0), ('images_test/1429.jpg', 1.0), ('images_test/101.jpg', 1.0), ('images_test/1660.jpg', 1.0), ('images_test/1993.jpg', 1.4142135623730951), ('images_test/1718.jpg', 1.4142135623730951), ('images_test/1209

In [20]:
train_to_test_dict = {}
for idx, image_file in enumerate(df_train['image_file'].tolist()):
    train_to_test_dict[image_file] = predictions[idx]